In [1]:
'''
Adding Glycosolation. 
Adding entropy. 
Clevage sites.
'''

'\nAdding Glycosolation. \nAdding entropy. \nClevage sites.\n'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:

df_epi = pd.read_csv('data/processed/parallelized_epitope_entropies_larger.csv')
df_epi.head()

,protein,epitope,start_pos,epi_len,entropy,perc_mutated
0,E,MYSFVSEE,0,8,0.035578,0.002893
1,E,YSFVSEET,1,8,0.042296,0.003441
2,E,SFVSEETG,2,8,0.042296,0.003441
3,E,FVSEETGT,3,8,0.041116,0.003363
4,E,VSEETGTL,4,8,0.038914,0.003206


In [5]:
df_cleave = pd.read_csv('data/processed/ProteinCleavageSites.csv')
df_cleave.head()

,Start,End,Sub_Protein,Protein
0,0,179,Non-structural protein 1,ORF1a
1,180,817,Non-structural protein 2,ORF1a
2,818,2762,Non-structural protein 3,ORF1a
3,2763,3262,Non-structural protein 4,ORF1a
4,3263,3568,3C-like proteinase,ORF1a


In [6]:
df_glyco = pd.read_csv('data/processed/glycosolation_sites_processed.csv')
df_glyco.head()


,protein,position,seq,prob_of_glyco
0,E,47,NVSL,0.6507
1,E,65,NSSR,0.6339
2,M,4,NGTI,0.7577
3,N,46,NNTA,0.6798
4,N,76,NSSP,0.2149


In [7]:
df_glyco.seq.apply(lambda x: len(x))

0     4
1     4
2     4
3     4
4     4
     ..
72    4
73    4
74    4
75    4
76    4
Name: seq, Length: 77, dtype: int64

In [8]:
df = pd.DataFrame(df_epi)

In [9]:
df.head()

,protein,epitope,start_pos,epi_len,entropy,perc_mutated
0,E,MYSFVSEE,0,8,0.035578,0.002893
1,E,YSFVSEET,1,8,0.042296,0.003441
2,E,SFVSEETG,2,8,0.042296,0.003441
3,E,FVSEETGT,3,8,0.041116,0.003363
4,E,VSEETGTL,4,8,0.038914,0.003206


In [10]:
# ignore the ending epitopes for ORF1a optional!!!
starts_to_drop = df[df.protein=='ORF1a'].groupby('epi_len').start_pos.max()
starts_to_drop

epi_len
8     4393
9     4392
10    4391
11    4390
13    4388
14    4387
15    4386
16    4385
17    4384
18    4383
19    4382
20    4381
21    4380
22    4379
23    4378
24    4377
25    4376
Name: start_pos, dtype: int64

In [11]:
indices = []
window_sizes = list(range(8,12)) + list(range(13,26))
for eplen, startpos in zip(window_sizes, starts_to_drop):
    indices.append( df[np.logical_and(df.protein=='ORF1a', 
                                      np.logical_and(df.start_pos==startpos,
                                                     df.epi_len==eplen ))].index[0])
indices

[16139,
 20532,
 24924,
 29315,
 33704,
 38092,
 42479,
 46865,
 51250,
 55634,
 60017,
 64399,
 68780,
 73160,
 77539,
 81917,
 86294]

In [12]:
len(df.epi_len.unique())

17

In [13]:
df.iloc[82889]

protein                             ORF1a
epitope         EEEQEEDWLDDDSQQTVGQQDGSED
start_pos                             971
epi_len                                25
entropy                          0.139463
perc_mutated                     0.010948
Name: 82889, dtype: object

In [14]:
df[np.logical_and(df.protein=='ORF1a', 
                    np.logical_and(df.start_pos==4376,
                    df.epi_len==25 ))].index[0]

86294

In [15]:
df.iloc[np.asarray(indices)].epitope.to_list()

['ADAQSFLN',
 'SADAQSFLN',
 'QSADAQSFLN',
 'LQSADAQSFLN',
 'PMLQSADAQSFLN',
 'EPMLQSADAQSFLN',
 'REPMLQSADAQSFLN',
 'LREPMLQSADAQSFLN',
 'QLREPMLQSADAQSFLN',
 'DQLREPMLQSADAQSFLN',
 'CDQLREPMLQSADAQSFLN',
 'SCDQLREPMLQSADAQSFLN',
 'CSCDQLREPMLQSADAQSFLN',
 'GCSCDQLREPMLQSADAQSFLN',
 'YGCSCDQLREPMLQSADAQSFLN',
 'GYGCSCDQLREPMLQSADAQSFLN',
 'KGYGCSCDQLREPMLQSADAQSFLN']

In [16]:
df.shape

(163813, 6)

In [17]:
df.drop(indices, axis=0, inplace=True) 
df.shape

(163796, 6)

In [18]:
#df['epi_len'] = df.sequence.apply(lambda x: len(x))

In [19]:
# apply glyco to the epitopes. 
# and also the cleavage

In [18]:
df['glyco_probs'] = np.empty((len(df), 0)).tolist()
for i in range(len(df_glyco)):
    print('index',i)

    protein_mask = df.protein==df_glyco.iloc[i].protein # select relevant protein
    seq_start = df['start_pos']
    seq_end = df['start_pos']+(df['epi_len']-1)
    glyco_start = df_glyco.iloc[i].position
    glyco_end = df_glyco.iloc[i].position+(len(df_glyco.iloc[i].seq)-1)
    in_region = np.logical_and(glyco_start >= seq_start,glyco_start <= seq_end)
    #if all 4 matter::: 
    #.    front_in_region = np.logical_and(glyco_start >= seq_start,glyco_start <= seq_end)
    #     end_in_region = np.logical_and(glyco_end >= seq_start,glyco_end <= seq_end)
    #.    in_region = np.logical_or(front_in_region, end_in_region)
    in_region_and_protein = np.logical_and(protein_mask,in_region)
    
    df.loc[in_region_and_protein,'glyco_probs'] = df[in_region_and_protein]['glyco_probs'].apply(lambda x: x + [df_glyco.iloc[i].prob_of_glyco])
    #apply protein mask and then epitope mask


index 0
index 1
index 2
index 3
index 4
index 5
index 6
index 7
index 8
index 9
index 10
index 11
index 12
index 13
index 14
index 15
index 16
index 17
index 18
index 19
index 20
index 21
index 22
index 23
index 24
index 25
index 26
index 27
index 28
index 29
index 30
index 31
index 32
index 33
index 34
index 35
index 36
index 37
index 38
index 39
index 40
index 41
index 42
index 43
index 44
index 45
index 46
index 47
index 48
index 49
index 50
index 51
index 52
index 53
index 54
index 55
index 56
index 57
index 58
index 59
index 60
index 61
index 62
index 63
index 64
index 65
index 66
index 67
index 68
index 69
index 70
index 71
index 72
index 73
index 74
index 75
index 76


In [19]:
df.shape

(163796, 7)

In [20]:
df.head()

,protein,epitope,start_pos,epi_len,entropy,perc_mutated,glyco_probs
0,E,MYSFVSEE,0,8,0.035578,0.002893,[]
1,E,YSFVSEET,1,8,0.042296,0.003441,[]
2,E,SFVSEETG,2,8,0.042296,0.003441,[]
3,E,FVSEETGT,3,8,0.041116,0.003363,[]
4,E,VSEETGTL,4,8,0.038914,0.003206,[]


In [21]:
df[np.logical_and(df.epi_len!=11, df.epi_len!=12)].shape

(154086, 7)

In [22]:
len(np.arange(8,26))

18

In [23]:
155012-16

154996

In [24]:
df.head()

,protein,epitope,start_pos,epi_len,entropy,perc_mutated,glyco_probs
0,E,MYSFVSEE,0,8,0.035578,0.002893,[]
1,E,YSFVSEET,1,8,0.042296,0.003441,[]
2,E,SFVSEETG,2,8,0.042296,0.003441,[]
3,E,FVSEETGT,3,8,0.041116,0.003363,[]
4,E,VSEETGTL,4,8,0.038914,0.003206,[]


In [25]:
len_dropped = df[np.logical_and(df.epi_len!=11, df.epi_len!=12)]
len_dropped.glyco_probs.apply(lambda x: 1.0 if len(x)>0 else 0.0).sum()

18957.0

In [26]:
len_dropped = df[np.logical_and(df.epi_len!=11, df.epi_len!=12)]
len_dropped.glyco_probs.apply(lambda x: 1.0 if len(x)>0 else 0.0).sum()

18957.0

In [27]:
df.epi_len.unique()

array([ 8,  9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25])

In [28]:
#76 glyco sites. 
#there should be 
(np.arange(8,26).sum()-12-11)*76 # yep seems pretty reasonable

20824

In [29]:
18957.0/154996

0.12230638210018323

In [30]:
# binarizing glyco probs

df['glyco_probs'] = df.glyco_probs.apply(lambda x: 1.0 if len(x)>0 else 0.0)

In [31]:
df_glyco.head()

,protein,position,seq,prob_of_glyco
0,E,47,NVSL,0.6507
1,E,65,NSSR,0.6339
2,M,4,NGTI,0.7577
3,N,46,NNTA,0.6798
4,N,76,NSSP,0.2149


In [33]:
df.head()

,protein,epitope,start_pos,epi_len,entropy,perc_mutated,glyco_probs
0,E,MYSFVSEE,0,8,0.035578,0.002893,0.0
1,E,YSFVSEET,1,8,0.042296,0.003441,0.0
2,E,SFVSEETG,2,8,0.042296,0.003441,0.0
3,E,FVSEETGT,3,8,0.041116,0.003363,0.0
4,E,VSEETGTL,4,8,0.038914,0.003206,0.0


In [34]:
df[np.logical_and(df.protein=='E', np.logical_and(df.start_pos==6,df.epi_len==10 ))]

,protein,epitope,start_pos,epi_len,entropy,perc_mutated,glyco_probs
141,E,EETGTLIVNS,6,10,0.016271,0.001251,0.0


In [35]:
df.protein.unique()

array(['E', 'M', 'N', 'ORF10', 'ORF1a', 'ORF1b', 'ORF3a', 'ORF6', 'ORF7a',
       'ORF7b', 'ORF8', 'ORF9b', 'S'], dtype=object)

In [36]:
df_cleave.head()

,Start,End,Sub_Protein,Protein
0,0,179,Non-structural protein 1,ORF1a
1,180,817,Non-structural protein 2,ORF1a
2,818,2762,Non-structural protein 3,ORF1a
3,2763,3262,Non-structural protein 4,ORF1a
4,3263,3568,3C-like proteinase,ORF1a


In [37]:
# adding in the cleavage sites. End is the position right before the start of cleavage. 
df_cleave

,Start,End,Sub_Protein,Protein
0,0,179,Non-structural protein 1,ORF1a
1,180,817,Non-structural protein 2,ORF1a
2,818,2762,Non-structural protein 3,ORF1a
3,2763,3262,Non-structural protein 4,ORF1a
4,3263,3568,3C-like proteinase,ORF1a
5,3569,3858,Non-structural protein 6,ORF1a
6,3859,3941,Non-structural protein 7,ORF1a
7,3942,4139,Non-structural protein 8,ORF1a
8,4140,4252,Non-structural protein 9,ORF1a
9,4253,4391,Non-structural protein 10,ORF1a


In [38]:
df_cleave.Sub_Protein.unique()

array(['Non-structural protein 1', 'Non-structural protein 2',
       'Non-structural protein 3', 'Non-structural protein 4',
       '3C-like proteinase', 'Non-structural protein 6',
       'Non-structural protein 7', 'Non-structural protein 8',
       'Non-structural protein 9', 'Non-structural protein 10',
       'Non-structural protein 11', 'RNA-directed RNA polymerase',
       'Helicase', 'Guanine-N7 methyltransferase',
       'Uridylate-specific endoribonuclease', "2'-O-methyltransferase",
       'S1', 'S2'], dtype=object)

In [39]:
df_cleave.shape

(18, 4)

In [40]:
len(df_cleave.Sub_Protein.unique())

18

In [41]:
df_cleave.Protein.unique()

array(['ORF1a', 'ORF1b', 'S'], dtype=object)

In [42]:
len(df.protein.unique()) - len(df_cleave.Protein.unique()) + len(df_cleave.Sub_Protein.unique())

28

In [43]:
df_cleave.tail()

,Start,End,Sub_Protein,Protein
13,1524,2050,Guanine-N7 methyltransferase,ORF1b
14,2051,2396,Uridylate-specific endoribonuclease,ORF1b
15,2397,2694,2'-O-methyltransferase,ORF1b
16,0,684,S1,S
17,685,1273,S2,S


In [44]:
df['crosses_cleavage'] = 0
for i in range(len(df_cleave)):
    print('index',i)

    protein_mask = df.protein==df_cleave.iloc[i].Protein
    seq_start = df['start_pos']
    seq_end = df['start_pos']+(df['epi_len']-1)
    
    cleave_position = df_cleave.iloc[i].End+1 # as it is the position to be avoided. 
    # while I have marked the end of the proteins
    
    # rather than an AA to avoid like with glyco. here it is a spot in between AAs. 
    in_region = np.logical_and(cleave_position > seq_start,
                               cleave_position <= seq_end) 
    # conservative here where
    # I dont let it exist on the position before cleavage either. 
    in_region_and_protein = np.logical_and(protein_mask,in_region)
    
    df.loc[in_region_and_protein,'crosses_cleavage'] = 1
    #apply protein mask and then epitope mask


index 0
index 1
index 2
index 3
index 4
index 5
index 6
index 7
index 8
index 9
index 10
index 11
index 12
index 13
index 14
index 15
index 16
index 17


In [45]:
df.head()

,protein,epitope,start_pos,epi_len,entropy,perc_mutated,glyco_probs,crosses_cleavage
0,E,MYSFVSEE,0,8,0.035578,0.002893,0.0,0
1,E,YSFVSEET,1,8,0.042296,0.003441,0.0,0
2,E,SFVSEETG,2,8,0.042296,0.003441,0.0,0
3,E,FVSEETGT,3,8,0.041116,0.003363,0.0,0
4,E,VSEETGTL,4,8,0.038914,0.003206,0.0,0


In [46]:
df[np.logical_and(df.protein=="S", df.crosses_cleavage==1)]
# df['crosses_cleavage']
# df

,protein,epitope,start_pos,epi_len,entropy,perc_mutated,glyco_probs,crosses_cleavage
143118,S,NSPRRARS,678,8,0.276892,0.038474,0.0,1
143119,S,SPRRARSV,679,8,0.263279,0.037301,0.0,1
143120,S,PRRARSVA,680,8,0.286651,0.039334,0.0,1
143121,S,RRARSVAS,681,8,0.074736,0.006412,0.0,1
143122,S,RARSVASQ,682,8,0.080655,0.006881,0.0,1
...,...,...,...,...,...,...,...,...
163244,S,PRRARSVASQSIIAYTMSLGAENSV,680,25,0.454565,0.057945,0.0,1
163245,S,RRARSVASQSIIAYTMSLGAENSVA,681,25,0.220743,0.022834,0.0,1
163246,S,RARSVASQSIIAYTMSLGAENSVAY,682,25,0.218389,0.022678,0.0,1
163247,S,ARSVASQSIIAYTMSLGAENSVAYS,683,25,0.219566,0.022756,0.0,1


In [47]:
df.tail()

,protein,epitope,start_pos,epi_len,entropy,perc_mutated,glyco_probs,crosses_cleavage
163808,S,KGCCSCGSCCKFDEDDSEPVLKGVK,1244,25,0.116781,0.009853,0.0,0
163809,S,GCCSCGSCCKFDEDDSEPVLKGVKL,1245,25,0.113617,0.009618,0.0,0
163810,S,CCSCGSCCKFDEDDSEPVLKGVKLH,1246,25,0.113617,0.009618,0.0,0
163811,S,CSCGSCCKFDEDDSEPVLKGVKLHY,1247,25,0.099336,0.008367,0.0,0
163812,S,SCGSCCKFDEDDSEPVLKGVKLHYT,1248,25,0.098158,0.008289,0.0,0


In [48]:
df_cleave.head()

,Start,End,Sub_Protein,Protein
0,0,179,Non-structural protein 1,ORF1a
1,180,817,Non-structural protein 2,ORF1a
2,818,2762,Non-structural protein 3,ORF1a
3,2763,3262,Non-structural protein 4,ORF1a
4,3263,3568,3C-like proteinase,ORF1a


In [49]:
df_cleave.tail(10)

,Start,End,Sub_Protein,Protein
8,4140,4252,Non-structural protein 9,ORF1a
9,4253,4391,Non-structural protein 10,ORF1a
10,4392,4404,Non-structural protein 11,ORF1a
11,-9,922,RNA-directed RNA polymerase,ORF1b
12,923,1523,Helicase,ORF1b
13,1524,2050,Guanine-N7 methyltransferase,ORF1b
14,2051,2396,Uridylate-specific endoribonuclease,ORF1b
15,2397,2694,2'-O-methyltransferase,ORF1b
16,0,684,S1,S
17,685,1273,S2,S


In [50]:
df[np.logical_and(df.protein=='ORF1b', np.logical_and(df.start_pos==914,df.epi_len==10 ))]

,protein,epitope,start_pos,epi_len,entropy,perc_mutated,glyco_probs,crosses_cleavage
92584,ORF1b,MYTPHTVLQA,914,10,0.011111,0.000782,0.0,1


In [51]:
df[np.logical_and(df.epi_len!=11, df.epi_len!=12)].shape

(154086, 8)

In [52]:
df.epi_len.unique()

array([ 8,  9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25])

In [53]:
df.head()

,protein,epitope,start_pos,epi_len,entropy,perc_mutated,glyco_probs,crosses_cleavage
0,E,MYSFVSEE,0,8,0.035578,0.002893,0.0,0
1,E,YSFVSEET,1,8,0.042296,0.003441,0.0,0
2,E,SFVSEETG,2,8,0.042296,0.003441,0.0,0
3,E,FVSEETGT,3,8,0.041116,0.003363,0.0,0
4,E,VSEETGTL,4,8,0.038914,0.003206,0.0,0


In [54]:
len_dropped = df[np.logical_and(df.epi_len!=11, df.epi_len!=12)]
len_dropped.crosses_cleavage.sum()

3739

In [55]:
df.shape

(163796, 8)

In [56]:
len_dropped.crosses_cleavage.sum()/154996

0.02412320317943689

In [57]:
#18 cleavage sites. 
#there should be 
(np.arange(8,26).sum()-12-11)*18 # yep seems pretty reasonable

4932

In [58]:
df.crosses_cleavage.sum() # used to be 3888

3887

In [59]:
df.shape # used to be 164786,8 I believe 

(163796, 8)

In [60]:
'''Renaming Spike epitopes into S1 and S2'''
#df = pd.read_csv('../data/processed/AllEpitopeFeatures.csv')
#df.head()

'Renaming Spike epitopes into S1 and S2'

In [61]:
df_cleave.head()

,Start,End,Sub_Protein,Protein
0,0,179,Non-structural protein 1,ORF1a
1,180,817,Non-structural protein 2,ORF1a
2,818,2762,Non-structural protein 3,ORF1a
3,2763,3262,Non-structural protein 4,ORF1a
4,3263,3568,3C-like proteinase,ORF1a


In [62]:

furin_site = 684
s1_mask = np.logical_and( df.protein=='S', df['start_pos'] <= furin_site ) 
# some of these at the end this is not accurate but they will be filtered by cleavage anyways. 
df.loc[s1_mask, 'protein'] = 'S1'
s2_mask = np.logical_and( df.protein=='S', df['start_pos'] > furin_site )
df.loc[s2_mask, 'protein'] = 'S2'
df.head()

,protein,epitope,start_pos,epi_len,entropy,perc_mutated,glyco_probs,crosses_cleavage
0,E,MYSFVSEE,0,8,0.035578,0.002893,0.0,0
1,E,YSFVSEET,1,8,0.042296,0.003441,0.0,0
2,E,SFVSEETG,2,8,0.042296,0.003441,0.0,0
3,E,FVSEETGT,3,8,0.041116,0.003363,0.0,0
4,E,VSEETGTL,4,8,0.038914,0.003206,0.0,0


In [63]:
df.tail()

,protein,epitope,start_pos,epi_len,entropy,perc_mutated,glyco_probs,crosses_cleavage
163808,S2,KGCCSCGSCCKFDEDDSEPVLKGVK,1244,25,0.116781,0.009853,0.0,0
163809,S2,GCCSCGSCCKFDEDDSEPVLKGVKL,1245,25,0.113617,0.009618,0.0,0
163810,S2,CCSCGSCCKFDEDDSEPVLKGVKLH,1246,25,0.113617,0.009618,0.0,0
163811,S2,CSCGSCCKFDEDDSEPVLKGVKLHY,1247,25,0.099336,0.008367,0.0,0
163812,S2,SCGSCCKFDEDDSEPVLKGVKLHYT,1248,25,0.098158,0.008289,0.0,0


In [64]:
df_cleave.head()

,Start,End,Sub_Protein,Protein
0,0,179,Non-structural protein 1,ORF1a
1,180,817,Non-structural protein 2,ORF1a
2,818,2762,Non-structural protein 3,ORF1a
3,2763,3262,Non-structural protein 4,ORF1a
4,3263,3568,3C-like proteinase,ORF1a


In [65]:
df[np.logical_and(df.protein=='ORF1a', np.logical_and(df.start_pos==170,df.epi_len==10 ))]

,protein,epitope,start_pos,epi_len,entropy,perc_mutated,glyco_probs,crosses_cleavage
20703,ORF1a,RELMRELNGG,170,10,0.005898,0.000391,0.0,0


In [66]:
df.protein.unique()

array(['E', 'M', 'N', 'ORF10', 'ORF1a', 'ORF1b', 'ORF3a', 'ORF6', 'ORF7a',
       'ORF7b', 'ORF8', 'ORF9b', 'S1', 'S2'], dtype=object)

In [67]:
(df.perc_mutated==0).sum()

241

In [70]:
df.to_csv('data/processed/AllEpitopeFeatures_larger.csv', index=False)

In [73]:
# df

In [74]:
# trimming the very end of orf1b
df.head()


,protein,epitope,start_pos,epi_len,entropy,perc_mutated,glyco_probs,crosses_cleavage
0,E,MYSFVSEE,0,8,0.035578,0.002893,0.0,0
1,E,YSFVSEET,1,8,0.042296,0.003441,0.0,0
2,E,SFVSEETG,2,8,0.042296,0.003441,0.0,0
3,E,FVSEETGT,3,8,0.041116,0.003363,0.0,0
4,E,VSEETGTL,4,8,0.038914,0.003206,0.0,0


In [75]:
df.protein=='ORF1a'

0         False
1         False
2         False
3         False
4         False
          ...  
163808    False
163809    False
163810    False
163811    False
163812    False
Name: protein, Length: 163796, dtype: bool

In [76]:
df.shape # used to be 164786,8

(163796, 8)

In [77]:
df.epi_len.unique()

array([ 8,  9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25])